In [ ]:
# For licensing see accompanying LICENSE file.
# Copyright (C) 2025 Apple Inc. All Rights Reserved.
# Notebook to get human annotation data for longfact pairwise as csv

import pandas as pd

df_1 = pd.read_csv("../../data/human/arduin-long-form-factuality-20240806-2255_(uep-119589).csv")
df_2 = pd.read_csv("../../data/human/arduin-long-form-factuality-20240813-20240814-1240_(uep-120627).csv")

In [ ]:
df_1.columns = [column.replace("text_a", "text_1").replace("text_b", "text_2") for column in df_1.columns]

In [ ]:
df = pd.concat([df_1,df_2])

In [ ]:
def get_human_pref(row):
    rating = row["Satisfaction OPR text_2,text_1"]
    if rating < 0:
        return "text_2"
    elif rating > 0:
        return "text_1"
    else:
        return "tie"
    
def get_original_index(row):
    return int(row["Prompt Id"].split("-")[-1])

df["converted_ratings"] = df.apply(get_human_pref, axis=1)
df["original_index"] = df.apply(get_original_index, axis=1)

In [ ]:
df_orig = pd.read_csv("../../data/generated/longfact/individual_generations_v8_fixflips/longfact_v9.csv")
df_orig.head(2)

In [ ]:
full_df = df_orig.merge(df, left_on="index", right_on="original_index")

In [ ]:
full_df.columns

In [ ]:
def convert_human_pref(row):
    human_pref = row['converted_ratings']

    text_1 = row['Response - text_1']
    text_2 = row['Response - text_2']

    text_a = row['text_a']
    text_b = row['text_b']

    assert text_1 in [text_a,text_b]
    assert text_2 in [text_a,text_b]

    if text_a == text_1:
        # text_a corresponds to text_1, text_b to text_2
        return human_pref.replace("1","a").replace("2","b")
    elif text_a == text_2:
        # flipped correspondance
        return human_pref.replace("1","b").replace("2","a")
    else:
        raise ValueError()
    

full_df["human_pref"] = full_df.apply(convert_human_pref, axis=1)


In [ ]:
filt_df = full_df[full_df["human_pref"] != "tie"]
(filt_df["human_pref"] == filt_df["preferred_text"]).value_counts()

In [ ]:
full_df["human_annotation_strength"] = full_df["Satisfaction OPR text_2,text_1"]

In [ ]:
save_df = full_df[['index', 'prompt', 'text_a', 'text_b', 'original_text',
       'alt_text_original', 'preferred_text','human_pref', "human_annotation_strength"]]

In [ ]:
import pathlib

save_path = pathlib.Path("../../data/generated/longfact/individual_generations_v8_fixflips")
save_df.to_csv(save_path / "longfact_v9_incl_human.csv")
full_df.to_csv(save_path / "longfact_v9_incl_human_complete.csv")